In [2]:
import sys
import melee
import random

In [ ]:
# REPLACE WITH YOUR PATH NAME
pathToSlp = "replays\Game_20250422T225246.slp"

In [ ]:
console = melee.Console(system="file", path=pathToSlp)
console.connect()

while True:
    gamestate = console.step()
    # step() returns None when the file ends
    if gamestate is None:
        break
    print(gamestate.players[1].position.x, gamestate.players[1].position.y)

-38.8 35.2
-38.8 35.2
-38.8 35.2
-38.8 35.2
-38.8 35.2
-38.8 35.244923
-38.8 35.28984
-38.8 35.334763
-38.8 35.37968
-38.8 35.424603
-38.8 35.469524
-38.8 35.514442
-38.8 35.559364
-38.8 35.604282
-38.8 35.649204
-38.8 35.694126
-38.8 35.739044
-38.8 35.783966
-38.8 35.828884
-38.8 35.873806
-38.8 35.918728
-38.8 35.963646
-38.8 36.008568
-38.8 36.053486
-38.8 36.098408
-38.8 36.14333
-38.8 36.188248
-38.8 36.23317
-38.8 36.278088
-38.8 36.32301
-38.8 36.36793
-38.8 36.41285
-38.8 36.45777
-38.8 36.50269
-38.8 36.54761
-38.8 36.50269
-38.8 36.45777
-38.8 36.41285
-38.8 36.36793
-38.8 36.32301
-38.8 36.278088
-38.8 36.23317
-38.8 36.188248
-38.8 36.14333
-38.8 36.098408
-38.8 36.053486
-38.8 36.008568
-38.8 35.963646
-38.8 35.918728
-38.8 35.873806
-38.8 35.828884
-38.8 35.783966
-38.8 35.739044
-38.8 35.694126
-38.8 35.649204
-38.8 35.604282
-38.8 35.559364
-38.8 35.514442
-38.8 35.469524
-38.8 35.424603
-38.8 35.37968
-38.8 35.334763
-38.8 35.28984
-38.8 35.244923
-38.8 35.014923
-38.